In [ ]:
# !nvidia-smi

In [ ]:
# # Assign GPU number
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
# # !pip install tensorflow==2.5.0
# import tensorflow
# import tensorflow as tf
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

In [ ]:
# # !pip install silence-tensorflow
# from silence_tensorflow import silence_tensorflow
# silence_tensorflow()

In [ ]:
# config = ConfigProto()
# # config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

In [ ]:
# from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.utils import Sequence
# from sklearn.model_selection import train_test_split
# print(tf.test.is_gpu_available())

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/vaibhavkumar11/hindi-english-parallel-corpus')

Dataset URL: https://www.kaggle.com/datasets/vaibhavkumar11/hindi-english-parallel-corpus


100%|██████████| 112M/112M [00:00<00:00, 160MB/s] 


In [ ]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import pickle
import os
import re
import nltk
import joblib
import datetime
import time
import multiprocessing
from tqdm import tqdm
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.layers import Embedding,LSTM,Dense,BatchNormalization,Dropout,Bidirectional,Concatenate,LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping,TensorBoard,LearningRateScheduler
from tensorflow.keras.regularizers import l2
from nltk.translate import bleu_score
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.models import Model,load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dropout, Dense, Concatenate
from tensorflow.keras.models import Model

In [ ]:
train_final_data = pd.read_csv('/content/hindi-english-parallel-corpus/hindi_english_parallel.csv')
train_final_data = train_final_data[:10000]

In [ ]:
print(train_final_data.info())
print(train_final_data.dropna(inplace=True))
print(train_final_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   hindi    9997 non-null   object
 1   english  9996 non-null   object
dtypes: object(2)
memory usage: 156.4+ KB
None
None
<class 'pandas.core.frame.DataFrame'>
Index: 9993 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   hindi    9993 non-null   object
 1   english  9993 non-null   object
dtypes: object(2)
memory usage: 234.2+ KB
None


In [ ]:
train_data,val_data=train_test_split(train_final_data,test_size=0.2)
print(train_data.shape)
print(val_data.shape)

(7994, 2)
(1999, 2)


In [ ]:
train_data['hindi'] = ['<start> ']+train_data['hindi']+[' <end>']
train_data['english'] = ['<start> ']+train_data['english']+[' <end>']

In [ ]:
val_data['hindi'] = ['<start> ']+val_data['hindi']+[' <end>']
val_data['english'] = ['<start> ']+val_data['english']+[' <end>']

In [ ]:
val_data

,hindi,english
7114,<start> लोड <end>,<start> _ Load Log <end>
5915,<start> एक सूट <end>,<start> One Suit <end>
3985,<start> चार आधार <end>,<start> Four Foundations <end>
1410,<start> ऊपरी पटल के लिए खाका <end>,<start> The layout for the top panel <end>
8833,<start> input widget <end>,<start> Git: Pull complete. <end>
...,...,...
315,<start> अंतर्क्रियात्मक% s कार्रवाई करने योग्य...,<start> interactive% s is not actionable <end>
5439,<start> चिड़ी की दुक्की <end>,<start> two of clubs <end>
3402,<start> ताश बांटना बाकीः ~ a <end>,<start> Redeals left: ~ a <end>
863,<start> छवि (_ I) <end>,<start> _ Image <end>


In [ ]:
train_data

,hindi,english
331,<start> शिशु <end>,<start> Children <end>
7200,<start> 0 फाइल्स <end>,<start> Profile <end>
1966,<start> समयः <end>,<start> Time: <end>
1887,<start> हुकुम का दहला <end>,<start> ten of spades <end>
2398,<start> ब्लाक दस <end>,<start> Block Ten <end>
...,...,...
9874,<start> कोड समाप्ति सक्रिय करें <end>,<start> Min character for completion <end>
4258,<start> लाल पान की तिक्की <end>,<start> three of hearts <end>
5604,<start> हुकुम का दहला <end>,<start> the ten of spades <end>
796,<start> नीचे पैनल pluginview के लिए लेआउट. <end>,<start> The layout for the bottom panel plugin...


In [ ]:
#below is the arrangement for different tokenizers

In [ ]:
inp_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='',oov_token='<unk>')
inp_tokenizer.fit_on_texts(train_data.english)

inp_tokenizer.word_index
inp_vocab_size=len(inp_tokenizer.word_index)+1
print(inp_vocab_size)

out_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='',oov_token='<unk>')
out_tokenizer.fit_on_texts(train_data.hindi)
out_vocab_size = len(out_tokenizer.word_index)+1
print(out_vocab_size)

x_train_encoded = inp_tokenizer.texts_to_sequences(train_data.english)
y_train_encoded = out_tokenizer.texts_to_sequences(train_data.hindi)

x_val_encoded = inp_tokenizer.texts_to_sequences(val_data.english)
y_val_encoded = out_tokenizer.texts_to_sequences(val_data.hindi)

x_train_padded = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded,padding='post',maxlen=43)
y_train_padded = tf.keras.preprocessing.sequence.pad_sequences(y_train_encoded,padding='post',maxlen=43)
print(x_train_padded.shape,y_train_padded.shape)

x_val_padded = tf.keras.preprocessing.sequence.pad_sequences(x_val_encoded,padding='post',maxlen=43)
y_val_padded = tf.keras.preprocessing.sequence.pad_sequences(y_val_encoded,padding='post',maxlen=43)
print(x_val_padded.shape,x_val_padded.shape)

1924
2273
(7994, 43) (7994, 43)
(1999, 43) (1999, 43)


In [ ]:
train_data[:5]

,hindi,english
331,<start> शिशु <end>,<start> Children <end>
7200,<start> 0 फाइल्स <end>,<start> Profile <end>
1966,<start> समयः <end>,<start> Time: <end>
1887,<start> हुकुम का दहला <end>,<start> ten of spades <end>
2398,<start> ब्लाक दस <end>,<start> Block Ten <end>


In [ ]:
x_train_padded[0]

array([  2, 161,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0], dtype=int32)

In [ ]:
y_train_padded[0]

array([  2, 261,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0], dtype=int32)

In [ ]:
inp_tokenizer.word_index

{'<unk>': 1,
 '<start>': 2,
 '<end>': 3,
 'the': 4,
 'of': 5,
 '_': 6,
 'a': 7,
 'to': 8,
 'move': 9,
 '~': 10,
 'onto': 11,
 'remove': 12,
 's': 13,
 'project': 14,
 'for': 15,
 'clubs': 16,
 'diamonds': 17,
 'next': 18,
 'place': 19,
 'empty': 20,
 'hearts': 21,
 'file': 22,
 'card': 23,
 'spades': 24,
 'an': 25,
 'you': 26,
 'not': 27,
 'diamonds.': 28,
 'hearts.': 29,
 'clubs.': 30,
 'spades.': 31,
 'select': 32,
 'king': 33,
 'on': 34,
 'or': 35,
 'three': 36,
 'ten': 37,
 'no': 38,
 '%': 39,
 'four': 40,
 'five': 41,
 '+': 42,
 'plugin': 43,
 'game': 44,
 'two': 45,
 'eight': 46,
 'queen': 47,
 'and': 48,
 'are': 49,
 'cards': 50,
 'six': 51,
 'new': 52,
 'nine': 53,
 'in': 54,
 '-': 55,
 'seven': 56,
 'slot': 57,
 'ace': 58,
 'view': 59,
 'tableau': 60,
 'jack': 61,
 '/': 62,
 'name': 63,
 'deal': 64,
 'selected': 65,
 'highlight': 66,
 'add': 67,
 'show': 68,
 'all': 69,
 'searching': 70,
 'foundation': 71,
 'current': 72,
 'from': 73,
 'help': 74,
 'is': 75,
 'build': 76,
 'th

# sequential model

In [ ]:

inp_vocab_size = len(inp_tokenizer.word_index) + 1
print(inp_vocab_size)
out_vocab_size = len(out_tokenizer.word_index) + 1
print(out_vocab_size)

1924
2273


In [ ]:



inp_vocab_size = len(inp_tokenizer.word_index) + 1
out_vocab_size = len(out_tokenizer.word_index) + 1
embedding_dim = 300
units = 300

encoder_inputs = Input(shape=(None,), name='encoder_inputs')
encoder_emb = Embedding(input_dim=inp_vocab_size, output_dim=embedding_dim, mask_zero=True, name='encoder_embedding')(encoder_inputs)
encoder_lstm_output, encoder_state_h, encoder_state_c = LSTM(units, return_sequences=True, return_state=True, dropout=0.33, recurrent_initializer='glorot_uniform', name='encoder_lstm')(encoder_emb)
encoder_state_h = LayerNormalization(name='encoder_layer_norm_h')(encoder_state_h)
encoder_state_c = LayerNormalization(name='encoder_layer_norm_c')(encoder_state_c)


decoder_inputs = Input(shape=(None,), name='decoder_inputs')
decoder_emb = Embedding(input_dim=out_vocab_size, output_dim=embedding_dim, mask_zero=True, name='decoder_embedding')(decoder_inputs)
decoder_lstm_output, _, _ = LSTM(units, return_sequences=True, return_state=True, dropout=0.33, recurrent_initializer='glorot_uniform', name='decoder_lstm')(decoder_emb, initial_state=[encoder_state_h, encoder_state_c])
decoder_norm_output = LayerNormalization(name='decoder_layer_norm')(decoder_lstm_output)
decoder_drop = Dropout(0.33, name='decoder_dropout')(decoder_norm_output)
decoder_outputs = Dense(out_vocab_size, activation='softmax', name='decoder_dense')(decoder_drop)


model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)


# model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss='sparse_categorical_crossentropy')

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_embedding         │ (None, None, 300)      │        577,200 │ encoder_inputs[0][0]   │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_lstm (LSTM)       │ [(None, None, 300),    │        721,200 │ encoder_embedding[0][… │
│                           │ (None, 300), (None,    │                │ not_equal[0][0]        │
│                           │ 300)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_embedding         │ (None, None, 300)      │        681,900 │ decoder_inputs[0][0]   │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_layer_norm_h      │ (None, 300)            │            600 │ encoder_lstm[0][1]     │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_layer_norm_c      │ (None, 300)            │            600 │ encoder_lstm[0][2]     │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_lstm (LSTM)       │ [(None, None, 300),    │        721,200 │ decoder_embedding[0][… │
│                           │ (None, 300), (None,    │                │ encoder_layer_norm_h[… │
│                           │ 300)]                  │                │ encoder_layer_norm_c[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_layer_norm        │ (None, None, 300)      │            600 │ decoder_lstm[0][0]     │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_dropout (Dropout) │ (None, None, 300)      │              0 │ decoder_layer_norm[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_dense (Dense)     │ (None, None, 2273)     │        684,173 │ decoder_dropout[0][0]  │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,387,473 (12.92 MB)

 Trainable params: 3,387,473 (12.92 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
encoder_model = Model(encoder_inputs, [encoder_state_h, encoder_state_c])


inf_decoder_inputs = Input(shape=(None,), name="inf_decoder_inputs")
state_input_h = Input(shape=(units,), name="state_input_h")
state_input_c = Input(shape=(units,), name="state_input_c")

decoder_embedding_inf = model.get_layer('decoder_embedding')(inf_decoder_inputs)

decoder_lstm_output_inf, decoder_h, decoder_c = model.get_layer('decoder_lstm')(
    decoder_embedding_inf, initial_state=[state_input_h, state_input_c])

decoder_norm_output_inf = model.get_layer('decoder_layer_norm')(decoder_lstm_output_inf)

decoder_drop_inf = model.get_layer('decoder_dropout')(decoder_norm_output_inf)

decoder_outputs_inf = model.get_layer('decoder_dense')(decoder_drop_inf)

decoder_model = Model(inputs=[inf_decoder_inputs, state_input_h, state_input_c],
                      outputs=[decoder_outputs_inf, decoder_h, decoder_c])

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):

  #inverting the logic and ignoring losses from padding
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    #calculate loss considering masking
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
def data_generator(X,y,BATCH_SIZE,shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        dataset = dataset.shuffle(len(X)).batch(BATCH_SIZE,drop_remainder=True)
    else:
        dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
if not os.path.isdir('model_save_sequential(smart_compose)'):
    os.makedirs('model_save_sequential(smart_compose)')

In [ ]:


tf.keras.backend.clear_session()


BATCH_SIZE = 512
epochs = 20
learning_rate = 0.01


model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss=loss_function)


filepath = "model_save_sequential(smart_compose)/weights-{epoch:02d}-{loss:.4f}.weights.h5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, verbose=1, patience=2, min_lr=0.00001)
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, mode='min', restore_best_weights=True)

# log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)

callback_list = [reduce_lr, early_stop]


train = data_generator((x_train_padded, y_train_padded[:, :-1]), y_train_padded[:, 1:], BATCH_SIZE=BATCH_SIZE, shuffle=True)
val = data_generator((x_val_padded, y_val_padded[:, :-1]), y_val_padded[:, 1:], BATCH_SIZE=BATCH_SIZE, shuffle=False)


history = model.fit(train, validation_data=val, epochs=epochs, callbacks=callback_list)


# model.save(filepath="model_save", save_format="tf")


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:609: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 145ms/step - loss: 0.8216 - val_loss: 0.6273 - learning_rate: 0.0100
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 115ms/step - loss: 0.6385 - val_loss: 0.4889 - learning_rate: 0.0100
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 115ms/step - loss: 0.4950 - val_loss: 0.3758 - learning_rate: 0.0100
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step - loss: 0.3767 - val_loss: 0.2874 - learning_rate: 0.0100
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - loss: 0.2878 - val_loss: 0.2301 - learning_rate: 0.0100
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 127ms/step - loss: 0.2268 - val_loss: 0.1918 - learning_rate: 0.0100
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 123ms/step - loss: 0.1861 - val_loss: 0.1632 - learning_rate: 0.0100
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 117ms/step - loss: 0.1518 - val_loss: 0.1365 - learning_rate: 0.0100
Epoch 9/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - loss: 0.1214 - val_loss: 0.1132 - learning_rate: 0.0100
Epoch 10/20
15/15 ━━━

In [ ]:


tf.keras.backend.clear_session()


BATCH_SIZE = 512
epochs = 50
learning_rate = 0.001


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss=loss_function)


# filepath = "model_save_sequential(smart_compose)/weights-{epoch:02d}-{loss:.4f}.hdf5"
# checkpoint = ModelCheckpoint(filepath=filepath, monitor='loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, verbose=1, patience=2, min_lr=0.00001)
early_stop = EarlyStopping(monitor='loss', patience=8, verbose=1, mode='min', restore_best_weights=True)

# log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)

callback_list = [reduce_lr, early_stop]


train = data_generator((x_train_padded, y_train_padded[:, :-1]), y_train_padded[:, 1:], BATCH_SIZE=BATCH_SIZE, shuffle=True)
val = data_generator((x_val_padded, y_val_padded[:, :-1]), y_val_padded[:, 1:], BATCH_SIZE=BATCH_SIZE, shuffle=False)


history = model.fit(train, validation_data=val, epochs=epochs, callbacks=callback_list)


# model.save(filepath="model_save", save_format="tf")


Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 207ms/step - loss: 0.0230 - val_loss: 0.0730 - learning_rate: 0.0010
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 133ms/step - loss: 0.0186 - val_loss: 0.0725 - learning_rate: 0.0010
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 134ms/step - loss: 0.0178 - val_loss: 0.0723 - learning_rate: 0.0010
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 139ms/step - loss: 0.0166 - val_loss: 0.0719 - learning_rate: 0.0010
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 134ms/step - loss: 0.0155 - val_loss: 0.0721 - learning_rate: 0.0010
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.0151
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0009000000427477062.
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - loss: 0.0151 - val_loss: 0.0726 - learning_rate: 0.0010
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - loss: 0.0141 - val_loss: 0.0731 - learning_rate: 9.0000e-04
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.0145
Epoch 8: ReduceLROnPlateau 

In [ ]:



tf.keras.backend.clear_session()


BATCH_SIZE = 512
epochs = 20
learning_rate = 0.0001


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss=loss_function)


# filepath = "model_save_sequential(smart_compose)/weights-{epoch:02d}-{loss:.4f}.hdf5"
# checkpoint = ModelCheckpoint(filepath=filepath, monitor='loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, verbose=1, patience=2, min_lr=0.00001)
early_stop = EarlyStopping(monitor='loss', patience=8, verbose=1, mode='min', restore_best_weights=True)

# log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)

callback_list = [reduce_lr, early_stop]


train = data_generator((x_train_padded, y_train_padded[:, :-1]), y_train_padded[:, 1:], BATCH_SIZE=BATCH_SIZE, shuffle=True)
val = data_generator((x_val_padded, y_val_padded[:, :-1]), y_val_padded[:, 1:], BATCH_SIZE=BATCH_SIZE, shuffle=False)


history = model.fit(train, validation_data=val, epochs=epochs, callbacks=callback_list)


# model.save(filepath="model_save", save_format="tf")


Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 155ms/step - loss: 0.0098 - val_loss: 0.0761 - learning_rate: 1.0000e-04
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - loss: 0.0096 - val_loss: 0.0753 - learning_rate: 1.0000e-04
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - loss: 0.0095 - val_loss: 0.0754 - learning_rate: 1.0000e-04
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 0.0101
Epoch 4: ReduceLROnPlateau reducing learning rate to 8.999999772640876e-05.
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 119ms/step - loss: 0.0101 - val_loss: 0.0754 - learning_rate: 1.0000e-04
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 122ms/step - loss: 0.0099 - val_loss: 0.0754 - learning_rate: 9.0000e-05
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 122ms/step - loss: 0.0096 - val_loss: 0.0752 - learning_rate: 9.0000e-05
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - loss: 0.0096 - val_loss: 0.0759 - learning_rate: 9.0000e-05
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 0.0096
Epo

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
import os
import json
import tensorflow as tf


model_dir = "/content/model_save(final)"
lang_code = 'en'
date_of_training = '2024-07-12'

encoder_model_json_path = os.path.join(model_dir, 'encoder_' + str(lang_code) + '_' + str(date_of_training) + '.json')
decoder_model_json_path = os.path.join(model_dir, 'decoder_' + str(lang_code) + '_' + str(date_of_training) + '.json')
encoder_model_weights_path = os.path.join(model_dir, 'encoder_' + str(lang_code) + '_' + str(date_of_training) + '.weights.h5')
decoder_model_weights_path = os.path.join(model_dir, 'decoder_' + str(lang_code) + '_' + str(date_of_training) + '.weights.h5')
complete_model_path = os.path.join(model_dir, 'complete_model_' + str(lang_code) + '_' + str(date_of_training) + '.weights.h5')
encoder_tflite_path = os.path.join(model_dir, 'encoder_' + str(lang_code) + '_' + str(date_of_training) + '.tflite')
decoder_tflite_path = os.path.join(model_dir, 'decoder_' + str(lang_code) + '_' + str(date_of_training) + '.tflite')
print(encoder_model_json_path)

encoder_model_json = encoder_model.to_json()
decoder_model_json = decoder_model.to_json()

os.makedirs(model_dir, exist_ok=True)

print(encoder_model_json_path)

with open(encoder_model_json_path, "w") as json_file:
    json_file.write(encoder_model_json)

with open(decoder_model_json_path, "w") as json_file:
    json_file.write(decoder_model_json)


encoder_model.save_weights(encoder_model_weights_path)
decoder_model.save_weights(decoder_model_weights_path)


model.save(complete_model_path)

/content/model_save(final)/encoder_en_2024-07-12.json
/content/model_save(final)/encoder_en_2024-07-12.json


In [ ]:
encoder_model.load_weights('model_path')
decoder_model.load_weights('model_path')

In [ ]:
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_inputs (InputLayer)  [(None, None)]            0         
_________________________________________________________________
encoder_embedding (Embedding (None, None, 300)         2995500   
_________________________________________________________________
encoder_lstm (LSTM)          [(None, None, 300), (None 721200    
Total params: 3,716,700
Trainable params: 3,716,700
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_inputs (InputLayer)  [(None, None)]            0         
_________________________________________________________________
encoder_embedding (Embedding (None, None, 300)         2995500   
________________________________

In [ ]:
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inf_decoder_inputs (InputLayer) [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_embedding (Embedding)   (None, None, 300)    2996100     inf_decoder_inputs[0][0]         
__________________________________________________________________________________________________
state_input_h (InputLayer)      [(None, 300)]        0                                            
__________________________________________________________________________________________________
state_input_c (InputLayer)      [(None, 300)]        0                                            
____________________________________________________________________________________________

In [ ]:
def predict_sentence(encoder_model, decoder_model, input_sentence, inp_tokenizer, out_tokenizer, max_encoder_seq_length, max_decoder_seq_length):

    input_sentence = '<start> ' + input_sentence + ' <end>'
    input_sequence = inp_tokenizer.texts_to_sequences([input_sentence])
    input_sequence = tf.keras.preprocessing.sequence.pad_sequences(input_sequence, maxlen=max_encoder_seq_length, padding='post')
    states_value = encoder_model.predict(input_sequence)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = out_tokenizer.word_index['<start>']


    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:

        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)


        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = out_tokenizer.index_word[sampled_token_index]

        decoded_sentence += ' ' + sampled_char


        if (sampled_char == '<end>' or len(decoded_sentence) >= max_decoder_seq_length):
            stop_condition = True


        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index


        states_value = [h, c]

    return decoded_sentence


input_sentence = "Remove the six of hearts."
max_encoder_seq_length = 43
max_decoder_seq_length = 43
encoder_model.load_weights('../encoder_en_2024-07-12.h5')
decoder_model.load_weights('../decoder_en_2024-07-12.h5')
decoded_sentence = predict_sentence(encoder_model, decoder_model, input_sentence, inp_tokenizer, out_tokenizer, max_encoder_seq_length, max_decoder_seq_length)
print('Decoded sentence:', decoded_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Decoded sentence:  लाल पान का बादशाह हटाएँ. <end>
